In [ ]:
"""WOrking on CM1 dataset"""

In [1]:
# Loading CM1 folder
import pandas as pd
import os

# Loading folder path
CM1_path = r"C:\Local Disk (A)\Github Files\Projects\Mass-Sceptra-Classification\Datasets\CM1"
data = []

# Combining all the files in the CM1 folder
for file in os.listdir(CM1_path):
    if file.endswith('.csv'):
        file_path = os.path.join(CM1_path, file)
        df = pd.read_csv(file_path, skiprows=1) # skipping the first row
        # Append to the list of DataFrames
        data.append(df)
# Concatenate all DataFrames into a single DataFrame
combined_CM1_data = pd.concat(data, ignore_index=True)

print(combined_CM1_data.head(5))
print("Rows & columns:", combined_CM1_data.shape)

   #Point  X(Thompsons)    Y(Counts)
0       0          40.0    46.235340
1       1          40.9    58.811787
2       2          41.0   617.287781
3       3          41.2    62.857056
4       4          42.1  3057.952637
Rows & columns: (92310, 3)


In [2]:
# Preprocessing Data

# summary of the datset
print("Summary :")
summary = combined_CM1_data.describe()
print(summary)

# Cheking for missing values
print("")
print("Cheking for missing values :")
is_null = combined_CM1_data.isnull().sum()
print(is_null)

# dropping "#Point" column as it is not relevant
print("")
print("After droping '#Point' column :")
combined_CM1_data = combined_CM1_data.drop('#Point', axis= 1)
print(combined_CM1_data.head(5))

# Rounding up the values of X(Thompsons) column
print("")
print("After Rounding :")
import numpy as np
combined_CM1_data['X(Thompsons)'] = np.ceil(combined_CM1_data['X(Thompsons)'])
print(combined_CM1_data.head(5))

# Adding features
import numpy as np
from scipy.spatial.distance import cosine

# Creating a reference vector that is the mean of Y(Counts) repeated to match length
#reference_vector = [combined_CM1_data['Y(Counts)'].mean()] * len(combined_CM1_data)
# Calculate cosine similarity between each row and the reference vector
#combined_CM1_data['cosine_similarity'] = combined_CM1_data['Y(Counts)'].apply(
    #lambda y: 1 - cosine([y], reference_vector)
#)

# 2. Area Ratio: Area ratio using trapezoidal rule with respect to X(Thompsons)
print("\nCalculating Area Ratio...")
combined_CM1_data['area_ratio'] = combined_CM1_data['Y(Counts)'] / combined_CM1_data['Y(Counts)'].sum()

# 3. Standard Deviation Ratio: Standard deviation ratio between Y(Counts) and X(Thompsons)
print("\nCalculating Standard Deviation Ratio...")
combined_CM1_data['std_ratio'] = combined_CM1_data['Y(Counts)'] / (combined_CM1_data['X(Thompsons)'].std() + 1)  # +1 to avoid division by zero

# Display the DataFrame with the new features
print("\nAfter adding new features:")
print(combined_CM1_data.head())
# Normalizing the dataset
print("")
print("After Normalization :")
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

scaler = MinMaxScaler()
columns_to_normalize = ['X(Thompsons)', 'Y(Counts)', 'area_ratio', 'std_ratio']
combined_CM1_data[columns_to_normalize] = scaler.fit_transform(combined_CM1_data[columns_to_normalize])
print(combined_CM1_data.head(5))

Summary :
             #Point  X(Thompsons)      Y(Counts)
count  92310.000000  92310.000000   92310.000000
mean     171.147308    143.243818    1193.070302
std      107.140743     68.833009    9722.745794
min        0.000000     39.900000       0.003136
25%       82.000000     88.200000      22.855255
50%      164.000000    134.000000      70.941631
75%      248.000000    186.100000     283.324089
max      521.000000    413.200000  654898.562500

Cheking for missing values :
#Point          0
X(Thompsons)    0
Y(Counts)       0
dtype: int64

After droping '#Point' column :
   X(Thompsons)    Y(Counts)
0          40.0    46.235340
1          40.9    58.811787
2          41.0   617.287781
3          41.2    62.857056
4          42.1  3057.952637

After Rounding :
   X(Thompsons)    Y(Counts)
0          40.0    46.235340
1          41.0    58.811787
2          41.0   617.287781
3          42.0    62.857056
4          43.0  3057.952637

Calculating Area Ratio...

Calculating Standard Devi

In [ ]:
# Ramdom Forest on CM1
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Convert Y(Counts) into binary labels based on the median
y = (combined_CM1_data['Y(Counts)'] > combined_CM1_data['Y(Counts)'].median()).astype(int)

# Split dataset into features and target
X = combined_CM1_data[['X(Thompsons)', 'area_ratio', 'std_ratio']]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

# Initialize SVM Classifier with an RBF kernel
model = RandomForestClassifier(n_estimators=100, random_state=50)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")